<span style="font-width:bold; font-size: 3rem; color:#333;">- Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Parse Data
2. Feature Group Insertion

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import *
import warnings
warnings.filterwarnings("ignore")

/home/jdowling/anaconda3/envs/book/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get AQI API Token

Go here first, to get your AQI API key
https://aqicn.org/data-platform/token/

Then enter the token you copied into the following variable in the code below. 

AQI_API_KEY = 

In [2]:
# Getting the current date
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)

country="sweden"
city="stockholm"
street="stockholm-hornsgatan-108-gata"

project = hopsworks.login()
fs = project.get_feature_store() 

# Get your AQI_API_KEY from here:
# https://aqicn.org/data-platform/token/

#AQI_API_KEY = # PUT YOUR AQI API KEY HERE
AQI_API_KEY = hopsworks.connection().get_secrets_api().get_secret("AQI_API_KEY").value


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works:443/p/5240
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [3]:

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Get Yesterday's Air Quality data (PM2.5) from the Feature Group</span>
My Air quality sensor is:  https://aqicn.org/station/sweden/stockholm-sodermalm/ 

In [4]:
# Read all data from the feature group
df_air_quality = air_quality_fg.read()

df = pd.DataFrame()
df_air_quality['day'] = pd.to_datetime(df_air_quality['date']).dt.date

Finished: Reading data from Hopsworks, using Hive (2.25s) 


In [5]:
# Extracting the "date" and "city_name" columns from the 'df_air_quality' DataFrame
# Grouping the data by "city_name" and finding the maximum date for each city
last_dates_aq = df_air_quality[["date", "city", "street"]].groupby("street").max()

# Converting the date values to string format for consistency
last_dates_aq.date = last_dates_aq.date.astype(str)

In [6]:
import requests
import pandas as pd

# The API endpoint URL
url = f"https://api.waqi.info/feed/{country}/{street}/?token={AQI_API_KEY}"

# Make a GET request to fetch the data from the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the JSON content from the response
    data = response.json()
    
    # Check if the API response contains the data
    if data['status'] == 'ok':
        # Extract the air quality data
        aqi_data = data['data']
        aq_today_df = pd.DataFrame()  
        aq_today_df['pm25'] = [aqi_data['iaqi'].get('pm25', {}).get('v', None)]
        aq_today_df['pm25'] = aq_today_df['pm25'].astype('float32')
    else:
        print("Error: The API response does not contain data.")
else:
    print("Failed to retrieve data. Status Code:", response.status_code)
    
aq_today_df['country'] = country
aq_today_df['city'] = city
aq_today_df['street'] = street
aq_today_df['date'] = today
aq_today_df['date'] = pd.to_datetime(aq_today_df['date'])

aq_today_df

,pm25,country,city,street,date
0,18.0,sweden,stockholm,stockholm-hornsgatan-108-gata,2024-03-05


In [7]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float32(1), object(3)
memory usage: 164.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather data</span>

In [8]:
hourly_df = get_hourly_weather_forecast(city)
hourly_df = hourly_df.set_index('date')

## We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.19999694824219°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-03-06,4.15,0.0,6.725354,74.475830,stockholm
1,2024-03-07,6.70,0.0,7.895416,294.227722,stockholm
2,2024-03-08,3.20,0.0,11.609651,7.124930,stockholm
3,2024-03-09,0.30,0.0,7.100310,59.534538,stockholm
4,2024-03-10,0.80,0.0,15.281989,74.981590,stockholm
5,2024-03-11,2.50,0.0,11.609651,82.875084,stockholm
6,2024-03-12,5.55,0.0,8.049845,169.695206,stockholm
7,2024-03-13,5.45,0.0,13.979871,168.111359,stockholm
8,2024-03-14,3.85,0.2,15.496736,177.337051,stockholm
9,2024-03-15,9.90,0.1,14.336861,231.115494,stockholm


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [10]:
# Insert new data
air_quality_fg.insert(aq_today_df)

Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works:443/p/5240/fs/5188/fg/7232


Uploading Dataframe: 100.00% |█████████████████████████████████████████████████████████████████████████████████| Rows 1/1 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/5240/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fea4062eec0>,
 {
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "aq_expectation_suite",
     "run_id": {
       "run_time": "2024-03-05T21:03:35.975333+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "d514152a-db33-11ee-a6fa-00155dd09873"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20240305T210335.975219Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "result": {
         "observed_value": 18.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-0

In [11]:
# Insert new data
weather_fg.insert(daily_df)

Validation succeeded.
Validation Report saved successfully, explore a summary at https://snurran.hops.works:443/p/5240/fs/5188/fg/7233


Uploading Dataframe: 100.00% |███████████████████████████████████████████████████████████████████████████████| Rows 10/10 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://snurran.hops.works/p/5240/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fea4a02cd90>,
 {
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.15.12",
     "expectation_suite_name": "weather_expectation_suite",
     "run_id": {
       "run_time": "2024-03-05T21:03:41.386239+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "d84dba2a-db33-11ee-a6fa-00155dd09873"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20240305T210341.386116Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.15.12"
     }
   },
   "results": [
     {
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       },
       "result": {
         "observed_value": 6.725354194641113,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "valid

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
